[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CCS-ZCU/pribehy-dat/blob/master/scripts/site.ipynb)

Tento soubor je součástí sestavy elektronických studijních opor [Příběhy dat: Výpočetní přístupy ke studiu kultury a společnosti](https://github.com/CCS-ZCU/pribehy-dat/tree/master). 

# Formální síťová analýza

**autor**: *Vojtěch Kaše* (kase@ff.zcu.cz)

[![](https://ccs.zcu.cz/wp-content/uploads/2021/10/cropped-ccs-logo_black_space_240x240.png)](https://ccs.zcu.cz)

## Úvod a cíle kapitoly

V tomto notebooku si budeme prakticky osvojovat koncepty síťové analýzy. Z veřejně dostupných dat si vytvoříme několik síťových grafů, které budeme dále upravovat, analyzovat a vizualizovat.

Jedním z nejhodnotnějších typů historických dat jsou sbírky dopisů, které nám umožňují sledovat kdo, s kým a kdy udřžoval kontakty. Řada těchto dopisních sbírek byla v posledních dekádách digitalizována. Existují tak například digitalizované kolekce sbírkek dopisů středověkých žen (https://epistolae.ctl.columbia.edu/letters/) nebo rozsáhlá kolekce raně novověkých dopisů EMLO (=Early Modern Letters Online, http://emlo-portal.bodleian.ox.ac.uk). Některé tyto datasety umožňují přístup pouze pomocí prohlížeče, a tudíž se nehodí pro datově analytickou práci. Jiné jsou naopak vzorovými příklady datového kurátorství. Ty zde budeme používat.

Konkrétně využijeme dataset dopisů mezi britskými vědci konce 18. a celého 19. století **Ɛpsilon** ([web](https://epsilon.ac.uk)), vyvíjený týmem z *Cambridge University Digital Library*. 
> Ɛpsilon opens up new research opportunities in the history of 19th century science by bringing correspondence data and transcriptions from multiple sources into a single cross-searchable digital platform. It currently holds details of over 50,000 letters and is growing. 

Alespoň z pohledu datové analýzy je velkou devízou tohoto projektu fakt, že veškerá data jsou dostupná nejen pro potřeby prohledávání a pročítání na webu projektu, ale také ve velice úhledné a praktické formě dostupná na GitHubu ([zde](https://github.com/cambridge-collection/epsilon-data)). Nachází se zde jak digitální edice každého jednotlivého dopisu podle standardu TEI-XML, tak i tabulky metadat ve formátu CSV. S těmi budeme níže pracovat my, když se je přímo z GitHubu načteme do našeho výpočetního prostředí. 

Nejprve budeme pracovat s kolekcí dopisů *Londínské Linneovské společnosti*, která byla založena roku 1788 a existuje dodnes  ([wikipedia](https://en.wikipedia.org/wiki/Linnean_Society_of_London)). Ač nese jméno významného švédského vědce Carla Linného ([wikipedia](https://cs.wikipedia.org/wiki/Carl_Linné)), otce vědecké taxonomie, tato vědecká společnost vznikla v Anglii až po jeho smrti.  

Tabulková data budeme zpracovávat pomocí knihovny **pandas**. K síťové analýze využijeme knihovnu **networkX**, jejíž dokumentaci doporučuji k projití si - [zde](https://networkx.org/documentation/stable/index.html)).

## Cvičení 1: Korespondence Linnevské společnosti

### Extrakce a přehled dat

In [ ]:
import numpy as np
import pandas as pd
import requests
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import regex
from bs4 import BeautifulSoup

In [ ]:
# navštívíme url adresu, kde jsou umístěny všechny csv soubory
resp_json = requests.get("https://api.github.com/repos/cambridge-collection/epsilon-data/contents/csv/").json()

Nyní si vypíšeme obsah načtených dat a zorientujeme v příslušné struktuře:

In [ ]:
resp_json

Vidíme, že ve struktuře je možné nalézt výpis jednotlivých `csv` souborů, které nás zajímají s odkazy na data ve formátu ke stažení (`"download_url"`)

In [ ]:
# vytvoříme si list URL adres všech csv souborů 
download_urls = [item["download_url"] for item in resp_json]
download_urls

In [ ]:
# a také list jmen všech těchto souborů
filenames = [item["name"] for item in resp_json]
filenames

In [ ]:
# načteme si data z jednoho konkrétního souboru
linnean = pd.read_csv("https://raw.githubusercontent.com/cambridge-collection/epsilon-data/main/csv/linnean-society.csv")
linnean.head()

Vidíme zde výpis prvních pěti řádek datové tabulky. Ale kolik vlastně tabulka čítá položek a kolik že je sloupců? To zjistíme z atributu `shape` (atributem je vlastnost datového objektu - jednou z vlastností datového objektu podle standardu `pd.DataFrame` je jeho tvar, tj. počet řádků a sloupců. 

In [ ]:
linnean.shape

Než se pustíme do síťových analýz, ještě si upravíme hodnoty v některých sloupcích tak, aby se nám s nimi dobře pracovalo. Sloupec `"sorting_date"` vyjadřuje dataci daného dopisu ve velice úhledném a srozumitelném formátu (yyyy-mm-dd). Jelikož jsme však naše data načetli z prostého `csv` souboru, Python neví nic o tom, že za touto řadou čísel a pomlček se jedná o dataci; k tomu jej musíme nainstruovat.

V buňce níže za tímto účelem vytváříme nový sloupec s výmluvným názvem `"datetime"`. Hodnoty v tomto sloupci jsou výsledkem použití (aplikování) funkce `to_datetime()` z knihovny pandas (`pd`) na hodnoty ve sloupci `"sorting_date"`. Tato funkce "přeloží" jednotlivá čísla na roky, měsíce a dny.

In [ ]:
linnean["datetime"] = linnean["sorting_date"].apply(pd.to_datetime)
linnean.head(5)

Ač hodnoty ve sloupci `"datetime"` vypadají stejně jako hodnoty ve sloupci `"sorting_date"`, chovají se odlišně. Umožňují nám přímo studovat časovou distribuci našich dat. Výhody tohoto formátu si všimneme, když na daný sloupec aplikujeme vizualizační metodu `hist()`:

In [ ]:
linnean["datetime"].hist()

In [ ]:
# snadno se můžeme např. podívat pouze na dopisy odeslané před začátkem 19. století
linnean["18thcent?"] = linnean["datetime"] < pd.to_datetime("1801-01-01")
# jen pro ověření se podívejme na prvních 5 řádek takto filtrovaných dat
linnean[linnean["18thcent?"]].head(5)

Vlastní jméno odesilatele a příjemce se nám rozpadá do vícero sloupců ("sender_surname", "sender_forename"). Vytvořme si nyní agregovanou podobu jména.

In [ ]:
linnean["sender_agr"] = linnean.apply(lambda row: str(row["sender_surname"]).replace(" ", "_") + "_" + str(row["sender_forename"]).replace(" ", "_"), axis=1)
linnean["recipient_agr"] = linnean.apply(lambda row: str(row["recipient_surname"]).replace(" ", "_") + "_" + str(row["recipient_forename"]).replace(" ", "_"), axis=1)

Nyní se podíváme na osoby, který poslaly a přijaly největší množství dopisů:

In [ ]:
linnean["sender_agr"].value_counts()

In [ ]:
linnean["recipient_agr"].value_counts()

V obou případech vidíme na prvním místě *Sira Jamese Edwarda Smithe*. Což, víme-li něco o Linneovské společnosti nebo podíváme-li se na wikipedii, není příliš překvapivé: jedná se o samotného zakladatele a dlouholetého předsedu této společnosti (viz [wikipedia](https://en.wikipedia.org/wiki/James_Edward_Smith_(botanist))).

V druhé tabulce vidíme na třetím místě také jeho manželku, *Pleasance Smithovou*, která byla taktéž významnou osobností dobového dění (taktéž viz [wikipedie](https://en.wikipedia.org/wiki/Pleasance_Smith)).  

### Tvorba síťových dat
Pro potřeby následujících si naše data výrazně přeskupíme a přetvoříme do podoby *seznamu vážených vazeb*.

In [ ]:
linnean_edges = linnean.groupby(["sender_agr", "recipient_agr"]).size().reset_index()
linnean_edges.columns = ["sender_agr", "recipient_agr", "letters_n"]
linnean_edges.head()

Jednotkou pozorování (čili řádkou tabulky) nyní již není každý jednotlivý dopis, ale pár odesilatele a příjemce s informací, kolik odesilatel příjemci zaslal dopisů (viz sloupec `"letters_n"`). Tato data lze již v podstatě považovat za tabulku hran. Můžeme si je setřídit od těch s největší váhou (tj. s nejvyšším počtem dopisů poslaných daným směrem).

In [ ]:
linnean_edges.sort_values("letters_n", ascending=False)

Z těchto dat si nyní vytvoříme síťový objekt.

In [ ]:
G = nx.from_pandas_edgelist(linnean_edges, 'sender_agr', 'recipient_agr', 'letters_n', create_using=nx.DiGraph())

In [ ]:
type(G)

Základní vlastnosti, které nás o našem grafu zajímají jsou, kolik má uzlů a kolik má hran?

In [ ]:
G.number_of_nodes()

In [ ]:
G.number_of_edges()

Další užitečnou informací je, kolik mají uzle v průměru vazeb (tzv. avarege degree).

In [ ]:
sum(dict(G.degree).values()) / G.number_of_nodes()

Stejně tak zajímavé bude se podívat, které uzly mají nejvyšší *in-degree* (tj. vazeb do něj vstupujících) a *out-degree*
(tj. vazeb z něj vystupujících). Podívejme se na deset uzlů s nejvyšší hodnotou in-degree:

In [ ]:
sorted(dict(G.in_degree()).items(), key=lambda item: item[1], reverse=True)[:10]

Vidíme, že zcela ústřední pozici zde zaujímá *Sir James Edward Smith*, zakladatel a dlouholetý předseda společnosti. Hned na druhém místě se v jednom uzlu potkávají dopisy, jejichž adresát je neznámý. Nebude od věci tento uzel ze sítě zcela odstranit.

In [ ]:
G.remove_node("Unknown_nan")

Utvořený síťový graf si můžeme bezprostřdně vizualizovat pomocí funkce `nx.draw()`:

In [ ]:
nx.draw(G)

Bohužel vidíme, že výsledek vypadá spíše nevábně. Podle všeho se zde příliš mnoho uzlů poblíž středu. Vidíme, že vazby mají podobu šipek. Je tomu tak proto, že se jedná o tzv. směrový graf.

Abychom dosáhli lepších výsledků, přidáme do vizualizační funkce několik dodatečných parametrů

In [ ]:
my_color = "darkgreen" # vybereme jakoukoli jinou barvu odtud: https://matplotlib.org/stable/gallery/color/named_colors.html
nx.draw(G, node_size=20, node_color=my_color, pos=nx.kamada_kawai_layout(G))

In [ ]:
# Tato buňka slouží ke kontrole průchodu tímto cvičením. 
# Pokud toto cvičení plníte v rámci svých studijních povinností na ZČU, buňku spusťte a držte se instrukcí.
import requests
exec(requests.get("https://sciencedata.dk/shared/856b0a7402aa7c7258186a8bdb329bd3?download").text)
kontrola_pruchodu(ntb="site", arg1=my_color)

Uzly v  grafu se jmenují stejně jako korespondenti. Pomocí syntaxe níže se tak můžeme podívat na vlastnosti jednotlivých vazeb. 

In [ ]:
G["Smith_Sir_James_Edward"]["Macleay_Alexander"]

In [ ]:
G["Macleay_Alexander"]["Smith_Sir_James_Edward"]


Zde se dozvídáme, že zatímco Sir James Edward Smith poslal Alexanderu Macleayovi 102, v opačném směru jich šlo 74.

Pro některé typy analýz je praktičtější i smysluplnější pracovat s nesměrovým grafem. Vazba tak nezohledňuje směr příslušné korespondence a váha může odpovídat součtu vyměněných dopisů v obou směrech. Transformovat naši síť do této podoby vyžaduje několik řádek kódu, jimiž se zde nemusíme příliš zaobírat, důležitější je výsledek.

In [ ]:
to_remove = []
edges_met = []
for node1, node2 in G.edges():
    if (G.has_edge(node2, node1)) & ((node2, node1) not in edges_met):
        G[node1][node2]["letters_n"] = G[node1][node2]["letters_n"] + G[node2][node1]["letters_n"]
        to_remove.append((node2, node1))
    edges_met.append((node1, node2))

In [ ]:
for u,v in to_remove:
    G.remove_edge(u,v)

In [ ]:
G = G.to_undirected().copy()

In [ ]:
len(G.edges())

Zde nyní uvidíme, že v obou směrech je hodnota "letters_n" totožná:

In [ ]:
G["Smith_Sir_James_Edward"]["Macleay_Alexander"]

In [ ]:
G["Macleay_Alexander"]["Smith_Sir_James_Edward"]

In [ ]:
weighted_degrees = {}
for node in G.nodes():
    weighted_degrees[node] = G.degree(node, weight='letters_n')

In [ ]:
list(weighted_degrees.items())[:10]

In [ ]:
# tento degree učiníme atributem našich uzlů
nx.set_node_attributes(G, weighted_degrees, 'weighted_degree')

Nyní si vyjmeme pouze uzly, které mají stupeň (degree) alespoň roven 2, tj. uzly osob, kteří v našem datasetu vedly korespondenci s více než jednou osobou.

In [ ]:
node_list = [node for node in G.nodes if G.degree(node) >= 2]
len(node_list)


Ukazuje se, že takových uzlů je v našem datasetu relativně málo. Vypišme si jejich jména.

In [ ]:
node_list

Nyní tento seznam jmen využijeme k vymezení výseku z našeho grafu (nazveme si jej `Gsub`), který bude zahrnovat pouze tyto uzly. 

In [ ]:
Gsub = G.subgraph(node_list)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(9, 6), dpi=300, tight_layout=True)

# pro potřeby vizualizace si ještě definujeme šířku čar jednotlivých vazeb,vycházející z objemu vyměněných dopisů. 
edge_widths = [np.sqrt(d['letters_n']) / 2 for (u, v, d) in Gsub.edges(data=True)]


nx.draw(Gsub, with_labels=True, pos=nx.kamada_kawai_layout(Gsub), node_size=100, nodelist=node_list, width=edge_widths, ax=ax)

ax.set_xlim(-1.3, 1.3)

Z takovéto vizualizace již lze vypozorovat leccos.

## Cvičení 2: Britská vědecká korespondence dlouhého 19. století jako celek

### Extrace a předzpracování dat

Nyní se vrátíme na začátek. Projekt Ɛpsilon totiž hostí vícero kolekcí dopisů z podobného období a je na místě očekávat, že se osoby v těchto kolekcích budou alespoň částečně překrývat.

Vypišme si tedy nejprve jména csv souborů s metadaty k těmto kolekcím.

In [ ]:
resp_json = requests.get("https://api.github.com/repos/cambridge-collection/epsilon-data/contents/csv/").json()
download_urls = [item["download_url"] for item in resp_json]
download_urls

Nyní pomocí cyklu FOR načteme data ze všech těchto souborů a nakonec je spojíme do jednoho objektu type `pd.DataFrame`.

In [ ]:
dfs = [] # připrav prázdný seznam, který budeme následně postupně plnit daty z jednotlivých kolekcí 
for url in download_urls: # pro každý z našeho seznamu souborů:
    try: # zkus: jej načíst jako dataframe
        collection_df = pd.read_csv(url, on_bad_lines='skip')
        collection_df["source"] = url.rpartition("/")[2] # přidej tomuto dataframu nový sloupec "source", kde bude uvedeno jméno souboru, ze kterého pochází
        dfs.append(collection_df) # přidej do seznamu aktuální dataframe
    except: # pokud to nejde:
        print("failed: ", url) # vypiš jméno souboru, u kterého to nejde
epsilon = pd.concat(dfs) # spoj do jednoho všechny dataframy uvnitř seznamu dfs

In [ ]:
epsilon.head(5)

In [ ]:
# jak dlouhý je náš dataset?
len(epsilon)

In [ ]:
# stejně jako výše agregujme jména autorů a příjemců dopisů do podoby bez mezer a závorek
epsilon["sender_agr"] = epsilon.apply( lambda row: str(row["sender_surname"]).replace(" ", "_").partition(" (")[0] + "_" + str(row["sender_forename"]).replace(" ", "_").partition(" (")[0], axis=1)
epsilon["sender_agr"] = epsilon["sender_agr"].apply(lambda x: regex.sub("[^\p{L}_-]", "", x))

epsilon["recipient_agr"] = epsilon.apply( lambda row: str(row["recipient_surname"]).replace(" ", "_").partition(" (")[0] + "_" + str(row["recipient_forename"]).replace(" ", "_").partition(" (")[0], axis=1)
epsilon["recipient_agr"] = epsilon["recipient_agr"].apply(lambda x: regex.sub("[^\p{L}_-]", "", x))

# odstraníme neznáme odesilatele a příjemce
epsilon = epsilon[~epsilon.isin(["Unknown_nan", "AT_TO_LOOK", "nan_nan"]).any(axis=1)]


Díky attributu "source" se vždy můžeme podívat pouze na výsek dat z konkrétního zdroje:  

In [ ]:
epsilon[epsilon["source"]=="darwin-family-letters.csv"].head(5)

Vypišme si nejplodnější autory a nejpopulárnější příjemce:

In [ ]:
epsilon["sender_agr"].value_counts()

In [ ]:
epsilon["recipient_agr"].value_counts()

Tentokrát si data vazeb do nesměrové podoby převedeme ještě před vytvořením grafu.

In [ ]:
epsilon_temp = epsilon.apply(lambda row: pd.Series(sorted([str(row["sender_agr"]), str(row["recipient_agr"])])), axis=1)
epsilon_temp.columns = ["node1", "node2"]
epsilon_edges = epsilon_temp.groupby(["node1", "node2"]).size().reset_index()
epsilon_edges.columns = ["node1", "node2", "weight"]
epsilon_edges = epsilon_edges[epsilon_edges["node1"] != epsilon_edges["node2"]]
epsilon_edges.head(5)

Data v této podobě můžeme již neprodleně použít k tvorbě sítě váženého nesměrového grafu.

In [ ]:
G = nx.from_pandas_edgelist(epsilon_edges, 'node1', 'node2', 'weight')

Opět se nejprve podíváme, z kolika uzlů a kolika hran naše síť sestává:

In [ ]:
G.number_of_nodes()

In [ ]:
G.number_of_edges()

Z těchto dat lze také snadno vypočítat tzv. *average degree*:

In [ ]:
(2 * G.number_of_edges()) / G.number_of_nodes()

U grafu s takto velkým počtem uzlů se nezřídka stane, že se ukáže, že je ve skutečnosti tvořen několika oddělenými komponenty, čili že síť není zcela propojená.

In [ ]:
len(list(nx.connected_components(G)))

Ano, to je i náš případ zde, když máme co dočinění s grafem, který sestává z více než 160 komponentů.

Podívejme se, z kolika uzlů sestává deset největších komponentů:

In [ ]:
components_sorted = sorted(list(nx.connected_components(G)), key=len, reverse=True)
[len(comp) for comp in components_sorted][:10]

Vidíme, že většina uzlů je součástí největšího komponentu, druhý největší komponent sestává již pouze z 5 uzlů. S klidným svědomím se nyní zaměříme pouze na největší komponent naší sítě.

In [ ]:
len(components_sorted[0])

In [ ]:
# Omezíme se na největší komponent.
G = G.subgraph(list(components_sorted[0]))

In [ ]:
G.number_of_nodes() #zkontrolujeme, že se filtrace uzlů povedla

In [ ]:
(2 * G.number_of_edges()) / G.number_of_nodes()


Pro potřeby několika dalších vizualizací nyní všem uzlům v rámci této sítě přiřadíme pozici v prostoru na základě jejich strukturelního postavení. Přiřazení těchto pozic v případě sítě, která sestává z tisíců uzlů, může být výpočetně poměrně náročné a zabrat nějaký čas. Abychom se níže vyhnuli zbytečnému čekání, vypočteme si tyto pozice uzlů již zde a dále je budeme používat v několika vizualizacích po sobě.

In [ ]:
%%time
pos = nx.spring_layout(G)

In [ ]:
fig, ax = plt.subplots(figsize=(9,6), dpi=300)
nx.draw(G, node_size=10, node_color="darkgreen", pos=pos, ax=ax)

In [ ]:
# Tato buňka slouží ke kontrole průchodu tímto cvičením. 
# Pokud toto cvičení plníte v rámci svých studijních povinností na ZČU, buňku spusťte a držte se instrukcí.
kontrola_pruchodu(ntb="site", arg1="site2")

> Zde končí povinná část cvičení.

Tato síť již možná má některé zajímavé topografické vlastnosti, které si zaslouží bližší analytické ohledání.

### Metriky centrality

Jedna skupina populárních a užitečných algoritmů jsou tzv. *metriky centrality* uzlů či vazeb. Uveďme si dvě takové metriky s jejich anglickými názvy a krátkým vysvětlením  nejznámnější s jejich anglickými názvy:
* **degree centrality**: je definován počtem vazeb, které daný uzel má
* **closeness centrality**: součet vzdáleností nejkratších cest potřebných k dosažení všech ostatních uzlů uvnitř sítě.
* **betweenness centrality** (mezilehlost): Jak často se ten který uzel nachází na trase spojující nejkratší cestou jakékoli další uzly uvnitř sítě.  
* **PageRank centrality**: je určen mnohonásobně opakovanými náhodnými procházkami po síti. Velikost PageRank je určena množstvím návštěv daného uzlu při těchto procházkách. Tento algoritmus byl původně vyvinut vývojáři od společnosti Google pro určení důležitých webových stránek.

S degree centrality jsme již vlastně  pracovali, když jsme se u předchozí sítě omezili pouze na uzly s degree alespoň 2. Tato metrika je také nejsnáze srozumitelná a bude zajímavé si zde představit její výsledky pro potřeby srovnání s výsledky ostatních metrik.
Jelikož zde však pracujeme s relativně rozsáhlou sítí a náš společný čas je omezený, vyzkoušíme si nyní pouze algrotimus pro PageRank, který je výpočetně nejméně náročný.

In [ ]:
degree_centrality = nx.degree_centrality(G)
degree_top_nodes = sorted(degree_centrality.items(), key=lambda x:x[1], reverse=True)
degree_top_nodes[:10]

In [ ]:
pagerank_centrality = nx.pagerank(G, max_iter=10000)
pagerank_top_nodes = sorted(pagerank_centrality.items(), key=lambda x:x[1], reverse=True)
pagerank_top_nodes[:10]

In [ ]:
%%time
betweenness_centrality = nx.betweenness_centrality(G)
betweenness_top_nodes = sorted(betweenness_centrality.items(), key=lambda x:x[1], reverse=True)
betweenness_top_nodes[:10]

In [ ]:
degree_pagerank_comparison = []
for deg, page, betw in zip(degree_top_nodes, pagerank_top_nodes, betweenness_top_nodes):
    degree_pagerank_comparison.append([deg[0], page[0], betw[0]])
centr_comparison_df = pd.DataFrame(degree_pagerank_comparison)
centr_comparison_df.columns = ["degree_node", "pagerank_node", "betw_node"]
print(centr_comparison_df.head(20).round(2))

V čem je toto srovnání potenciálně zajímavé? Podíváme-li se na pravou stranu tabulky, tj. uzly s největší betweenness centralitou, vidíme, že zejména ve druhé desítce se nachází nemálo uzlů, se kterými se na levé straně (u degree centrality) v první dvacítce vůbec nesetkáváme:
Jinými slovy, jedná se o uzly, jejichž centralita v rámci sítě není živena výlučně množstvím vazeb, které uvnitř sítě mají, ale spíše specifickým strukturálním postavením.
Podívejme se tedy na stejná data ještě jiným způsobem a totiž vypišme si, na kolikáté pozici se dvacítka uzlů s nejvyšší beteweenness centrality nachází z hlediska degree centrality.

In [ ]:
for node in centr_comparison_df["betw_node"][:20]:
    print(node, " degree:", G.degree(node), "degree rank:", [el[0] + 1 for el in enumerate(degree_top_nodes) if el[1][0] == node][0], )

Podívejme se nyní čtyři osobnosti:
* Charles Lyell
* Francis Galton
* John Phillips
* John Lubbock

Jejich degree rank je ve srovnání s jejich betweenness relativně vysoký. Zdá se, že tedy uzly mají v rámci grafu strukturálně zajímovou pozici.

Vytvořme tedy novou vizualizaci, v rámci které zaostříme pozornost právě na 20 uzlů s největší *betweenness*. Tyto uzly vyobrazíme odlišnou barvou a stejnou barvou vyobrazíme i jejich jména. 

In [ ]:
special_nodes = centr_comparison_df["betw_node"][:20] #["Lyell_Charles", "Galton_Francis", "Phillips_John", "Lubbock_John"]
special_pos = dict([(node, pos[node]) for node in special_nodes])
labels = {node: node for node in special_nodes}

In [ ]:
fig, ax = plt.subplots(figsize=(24,18), dpi=300)
special_nodes_color = "darkorange"
nx.draw(G, node_size=10, node_color="black", edge_color="grey",pos=pos, ax=ax, alpha=0.5)
nx.draw_networkx_nodes(G, nodelist=special_nodes, node_size=50, node_color=special_nodes_color, pos=special_pos, ax=ax)
nx.draw_networkx_labels(G, font_color=special_nodes_color, pos=special_pos, labels=labels,ax=ax)

Aby byl text čitelný a graf přehledný, vizualizace výše je výrazně větší než ty předchozí. Uložíme si ji do samostatného souboru ve formátu `png`.

In [ ]:
try:
    fig.savefig("../figures/epsilon_betw.png") # pokud pracujeme s repozitoří jako celkem, včetně podlsožky "figures"
except:
    fig.savefig("epsilon_betw.png") # pokud pracujeme s notebookem samostatně, např. přes Google Colab 

### Detekce komunit

Další důležitou rodinou algoritmů jsou algoritmy pro detekování komunit, neboli shluků uzlů, které jsou mezi sebou provázány více, než z uzly z jejich okolí. Zde použijeme takzvanou Lovaňskou metodu (podle působiště výzkumníků, kteří ji vyvinuli [viz [wikipedia](https://en.wikipedia.org/wiki/Louvain_method)]). Tento algoritmus se snaží nalézt takové rozdělení uzlů do komunit, které maximalizuje poměr vazeb mezi uzly uvnitř těchto komunit oproti jejich vazbám směrem ven z těchto komunit.

In [ ]:
from networkx.algorithms import community
communities = nx.community.louvain_communities(G, seed=1)
len(communities)

Algoritmus identifikoval 16 komunit. Podívejme se nejprve, kolik jednotlivé komunity čítají uzlů:


In [ ]:
[len(com) for com in communities]

In [ ]:
cmap = plt.get_cmap('viridis')
colors = [cmap(i) for i in np.linspace(0, 1, len(communities))]

fig, ax = plt.subplots(figsize=(24,18), dpi=300)
nx.draw_networkx_edges(G, edge_color="grey",pos=pos, alpha=0.5, ax=ax)

for community, color in zip(communities, colors):
    special_pos = dict([(node, pos[node]) for node in list(community)])
    #nx.draw(G, node_size=10, node_color="black", edge_color="grey",pos=pos, ax=ax, alpha=0.5)
    nx.draw_networkx_nodes(G, nodelist=list(community), node_size=10, node_color=[color], pos=special_pos, ax=ax)
ax.axis('off')

Vidíme, že tento algoritmus tedy dokáže velice pěkně zachytit strukturální vlastnosti dané sítě. To je v případě rozsáhlých grafů velice užitečné. 

## Alternativní datová sada: CorrespSearch

Alternativně bychom celé cvičení mohli absolvovat za využití mnoha dalších datasetů. Jedním z nich je dataset dostupný přes API na platformě CorrespSearch ([web](https://correspsearch.net/en/home.html)).

In [ ]:
correspsearch = pd.read_csv("https://correspsearch.net/api/v2.0/csv.xql?", sep=";")
correspsearch.head(10)

In [ ]:
%%time
for n in range(2,30):
    page_df = pd.read_csv("https://correspsearch.net/api/v2.0/csv.xql?x=" + str(n), sep=";")
    correspsearch = pd.concat([correspsearch, page_df])
    if n in range(0,3000,100):
        print(n)
    if len(page_df) < 100:
        break

In [ ]:
len(correspsearch)

In [ ]:
correspsearch = correspsearch[correspsearch["sender"].notnull() & correspsearch["addressee"].notnull()]
len(correspsearch)